In [1]:
import numpy as np
import json
from keras.models import Model
from keras.layers import Input
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras import backend as K

Using TensorFlow backend.


In [2]:
def format_decimal(arr, places=8):
    return [round(x * 10**places) / 10**places for x in arr]

### pipeline 10

In [3]:
data_in_shape = (24, 24, 2)

conv_0 = Conv2D(5, 3, 3, activation='relu', border_mode='valid', subsample=(2, 2), dim_ordering='tf', bias=True)
bn_0 = BatchNormalization(mode=0, axis=-1, epsilon=1e-3)
conv_1 = Conv2D(4, 3, 3, activation='relu', border_mode='same', subsample=(1, 1), dim_ordering='tf', bias=True)
bn_1 = BatchNormalization(mode=0, axis=-1, epsilon=1e-3)
conv_2 = Conv2D(3, 3, 3, activation='relu', border_mode='same', subsample=(1, 1), dim_ordering='tf', bias=True)
bn_2 = BatchNormalization(mode=0, axis=-1, epsilon=1e-3)
pool_0 = AveragePooling2D(pool_size=(2, 2), strides=None, border_mode='valid', dim_ordering='tf')
conv_3 = Conv2D(4, 3, 3, activation='linear', border_mode='valid', subsample=(1, 1), dim_ordering='tf', bias=True)
bn_3 = BatchNormalization(mode=0, axis=-1, epsilon=1e-3)
conv_4 = Conv2D(2, 3, 3, activation='relu', border_mode='same', subsample=(1, 1), dim_ordering='tf', bias=True)
bn_4 = BatchNormalization(mode=0, axis=-1, epsilon=1e-3)
pool_1 = AveragePooling2D(pool_size=(2, 2), strides=None, border_mode='valid', dim_ordering='tf')

input_layer = Input(shape=data_in_shape)
x = conv_0(input_layer)
x = bn_0(x)
x = conv_1(x)
x = bn_1(x)
x = conv_2(x)
x = bn_2(x)
x = pool_0(x)
x = conv_3(x)
x = bn_3(x)
x = conv_4(x)
x = bn_4(x)
output_layer = pool_1(x)
model = Model(input=input_layer, output=output_layer)

np.random.seed(11000)
data_in = 2 * np.random.random(data_in_shape) - 1

# set weights to random (use seed for reproducibility)
weights = []
for i, w in enumerate(model.get_weights()):
    np.random.seed(11000 + i)
    if i % 6 == 5:
        # std should be positive
        weights.append(0.5 * np.random.random(w.shape))
    else:
        weights.append(np.random.random(w.shape) - 0.5)
model.set_weights(weights)

result = model.predict(np.array([data_in]))

print({
    'input': {'data': format_decimal(data_in.ravel().tolist()), 'shape': list(data_in_shape)},
    'weights': [{'data': format_decimal(weights[i].ravel().tolist()), 'shape': list(weights[i].shape)} for i in range(len(weights))],
    'expected': {'data': format_decimal(result[0].ravel().tolist()), 'shape': list(result[0].shape)}
})

{'weights': [{'shape': [3, 3, 2, 5], 'data': [-0.41301916, -0.06548232, -0.27373104, 0.03463608, -0.45993658, 0.4865354, -0.19660301, -0.15413411, 0.07321166, 0.4660147, 0.03787414, 0.28148745, -0.48925452, 0.03053849, 0.46427825, -0.3340367, 0.49428242, -0.11228813, -0.37041425, -0.07633279, -0.42790588, 0.15693558, 0.49303097, 0.19876782, 0.04950603, 0.05806109, 0.08003724, 0.22830494, 0.49597338, -0.2804305, 0.42939438, 0.24273274, -0.45638821, -0.02349228, 0.17600165, 0.49173199, -0.45981763, -0.30585715, -0.0128569, -0.30422846, -0.35850094, 0.13143687, 0.33597551, -0.12145981, 0.08471007, -0.36319988, 0.11638593, 0.43377726, -0.0844127, -0.04048088, 0.41488413, -0.19283021, 0.31081145, -0.40510823, 0.39260834, -0.41011489, 0.40955659, 0.4535928, 0.4187157, 0.32875932, 0.3839722, -0.22723331, 0.39733175, 0.27852637, -0.46359521, 0.06622387, -0.1612151, -0.02176895, 0.07599637, -0.33743003, 0.38440459, 0.37484185, 0.29285209, -0.16604515, 0.26451534, 0.34501492, 0.36699692, -0.0617